In [4]:
import os
import numpy as np
import cv2
from scipy import ndimage
from PIL import Image
import matplotlib.pyplot as plt
import skimage
from skimage import filters as skifilters
%matplotlib inline
import glob

In [5]:
def occlusion_removal(img,threshold=12,SE_radius=6):
    # Remove Dark Hair Occlusions in Dermatoscopic Images via LUV Color Space
    # Using cv2.cvtColor() method 
    # Using cv2.COLOR_RGB2LUV color space 
    luv       = cv2.cvtColor(img, cv2.COLOR_RGB2Luv)

    # Morphological Closing via Spherical SE
    # Using Elliptical shaped kernal
    kernel    = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(SE_radius,SE_radius))
    closing   = cv2.morphologyEx(luv, cv2.MORPH_CLOSE, kernel)

    # Generate Masks via Hysteresis Thresholding Difference Image in L Channel
    diffc      = closing[:,:,0]-luv[:,:,0]
    maskc      = (skifilters.apply_hysteresis_threshold(diffc,threshold,80)).astype(np.uint8)*255
    
    
    mask_3dc   = maskc[:,:,None] * np.ones(3,dtype=np.uint8)[None, None, :]
    basec      = cv2.bitwise_not(maskc)
    base_3dc   = basec[:,:,None] * np.ones(3,dtype=np.uint8)[None, None, :]

    # Restitch Preprocessed Image
    preimagec  = ((base_3dc/255)*luv).astype(np.uint8)
    postimagec = ((mask_3dc/255)*closing).astype(np.uint8)
    fullc      = preimagec + postimagec
    outputc    = cv2.cvtColor(fullc, cv2.COLOR_Luv2RGB)

    return outputc, maskc

In [6]:
mylist = [f for f in glob.glob("*.jpg")]

for imagefile in mylist:
    raw_img = cv2.imread(imagefile)
    sample = np.array(raw_img)
    hr,mask = occlusion_removal(sample)
    path = 'C:/Users/vipin/Machine Learning/Occlusion_Remove/Occlusion_Removed'
    cv2.imwrite(os.path.join(path , 'hr_'+imagefile), hr)
    
    